# Assignment 1: Preprocessing and Text Classification

Student Name: Jiahao Chen

Student ID: 1118749

# General Info

<b>Due date</b>: Monday, 27 March 2023 1pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 9% of mark for class (with 8% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See "Using Jupyter Notebook and Python" page on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, Gensim, Keras and Pytorch. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3.8</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=https://www.nltk.org/book/>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place. In regards to the use of artificial intelligence tools in the context of academic integrity, please see the university's statement <a href="https://academicintegrity.unimelb.edu.au/plagiarism-and-collusion/artificial-intelligence-tools-and-technologies">here</a>.

# Overview

In this homework, you'll be working with a collection of tweets. The task is to predict the geolocation (country) where the tweet comes from. This homework involves writing code to preprocess data and perform text classification.

# Preprocessing (4 marks)

**Instructions**: Download the data (as1-data.json) from Canvas and put it in the same directory as this iPython notebook. Run the code below to load the json data. This produces two objects, `x` and `y`, which contains a list of  tweets and corresponding country labels (it uses the standard [2 letter country code](https://www.iban.com/country-codes)) respectively. **No implementation is needed.**

In [1]:
import json

x = []
y = []
data = json.load(open("as1-data.json"))
for k, v in data.items():
    x.append(k)
    y.append(v)
    
print("Number of tweets =", len(x))
print("Number of labels =", len(y))
print("\nSamples of data:")
for i in range(10):
    print("Country =", y[i], "\tTweet =", x[i])
    
assert(len(x) == 943)
assert(len(y) == 943)

Number of tweets = 943
Number of labels = 943

Samples of data:
Country = us 	Tweet = @Addictd2Success thx u for following
Country = us 	Tweet = Let's just say, if I were to ever switch teams, Khalesi would be top of the list. #girlcrush
Country = ph 	Tweet = Taemin jonghyun!!! Your birits make me go~ http://t.co/le8z3dntlA
Country = id 	Tweet = depart.senior 👻 rapat perdana (with Nyayu, Anita, and 8 others at Ruang Aescullap FK Unsri Madang) — https://t.co/swRALlNkrQ
Country = ph 	Tweet = Done with internship with this pretty little lady!  (@ Metropolitan Medical Center w/ 3 others) [pic]: http://t.co/1qH61R1t5r
Country = gb 	Tweet = Wow just Boruc's clanger! Haha Sunday League stuff that, Giroud couldn't believe his luck! #clown
Country = my 	Tweet = I'm at Sushi Zanmai (Petaling Jaya, Selangor) w/ 5 others http://t.co/bcNobykZ
Country = us 	Tweet = Mega Fest!!!! Its going down🙏🙌  @BishopJakes
Country = gb 	Tweet = @EllexxxPharrell wow love the pic babe xx
Country = us 	Tweet = You 

### Question 1 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation (based on frequency). The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); (2) lowercase all words; (3) remove any word that does not contain any English letters in the alphabet (e.g. {_hello_, _#okay_, _abc123_} would be kept, but not {_123_, _!!_}) and (4) remove stopwords (based on NLTK `stopwords`). An empty tweet (after preprocessing) and its country label should be **excluded** from the output (`x_processed` and `y_processed`).

**Task**: Complete the `preprocess_data(data, labels)` function. The function takes **a list of tweets** and **a corresponding list of country labels** as input, and returns **two lists**. For the first list, each element is a bag-of-words representation of a tweet (represented using a python dictionary). For the second list, each element is a corresponding country label. Note that while we do not need to preprocess the country labels (`y`), we need to have a new output list (`y_processed`) because some tweets maybe removed after the preprocessing (due to having an empty set of bag-of-words).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [2]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import re
from collections import defaultdict

tt = TweetTokenizer()
stopwords = set(stopwords.words('english')) #note: stopwords are all in lowercase

def preprocess_data(data, labels):
    
    ###
    # Your answer BEGINS HERE
    ###
    xx = []
    yy = []

    for tweet, country in zip(data, labels):
        
        # Tokenize each tweet into individual word tokens and lowercase all words
        tokens = tt.tokenize(tweet.lower())

        # Remove stopwords any word that does not contain any English letters
        words = []
        for token in tokens:
            if token not in stopwords and re.search('[a-z]', token):
                words.append(token)
        
        # Exclude empty tweets
        if len(words) == 0:
            continue

        # Create a bag-of-words representation based on frequency
        bow = defaultdict(int)
        for word in words:
            bow[word] += 1
        
        xx.append(bow)
        yy.append(country)

    return xx, yy
    ###
    # Your answer ENDS HERE
    ###

x_processed, y_processed = preprocess_data(x, y)

print("Number of preprocessed tweets =", len(x_processed))
print("Number of preprocessed labels =", len(y_processed))
print("\nSamples of preprocessed data:")
for i in range(10):
    print("Country =", y_processed[i], "\tTweet =", x_processed[i])

Number of preprocessed tweets = 943
Number of preprocessed labels = 943

Samples of preprocessed data:
Country = us 	Tweet = defaultdict(<class 'int'>, {'@addictd2success': 1, 'thx': 1, 'u': 1, 'following': 1})
Country = us 	Tweet = defaultdict(<class 'int'>, {"let's": 1, 'say': 1, 'ever': 1, 'switch': 1, 'teams': 1, 'khalesi': 1, 'would': 1, 'top': 1, 'list': 1, '#girlcrush': 1})
Country = ph 	Tweet = defaultdict(<class 'int'>, {'taemin': 1, 'jonghyun': 1, 'birits': 1, 'make': 1, 'go': 1, 'http://t.co/le8z3dntla': 1})
Country = id 	Tweet = defaultdict(<class 'int'>, {'depart.senior': 1, 'rapat': 1, 'perdana': 1, 'nyayu': 1, 'anita': 1, 'others': 1, 'ruang': 1, 'aescullap': 1, 'fk': 1, 'unsri': 1, 'madang': 1, 'https://t.co/swrallnkrq': 1})
Country = ph 	Tweet = defaultdict(<class 'int'>, {'done': 1, 'internship': 1, 'pretty': 1, 'little': 1, 'lady': 1, 'metropolitan': 1, 'medical': 1, 'center': 1, 'w': 1, 'others': 1, 'pic': 1, 'http://t.co/1qh61r1t5r': 1})
Country = gb 	Tweet = defau

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cjhm0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**For your testing**:

In [3]:
assert(len(x_processed) == len(y_processed))
assert(len(x_processed) > 800)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [4]:
def get_all_hashtags(data):
    hashtags = set([])
    for d in data:
        for word, frequency in d.items():
            if word.startswith("#") and len(word) > 1:
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(x_processed)
print("Number of hashtags =", len(hashtags))
print(sorted(hashtags))

Number of hashtags = 425
['#100percentpay', '#1stsundayofoctober', '#1yearofalmostisneverenough', '#2011prdctn', '#2015eebritishfilmacademyawards', '#2k16', '#2littlebirds', '#365picture', '#5sosacousticatlanta', '#5sosfam', '#8thannualpubcrawl', '#affsuzukicup', '#aflpowertigers', '#ahimacon14', '#aim20', '#airasia', '#allcity', '#alliswell', '#allwedoiscurls', '#amazing', '#anferneehardaway', '#ariona', '#art', '#arte', '#artwork', '#ashes', '#asian', '#asiangirl', '#askcrawford', '#askherforfback', '#askolly', '#asksteven', '#at', '#australia', '#awesome', '#awesomepict', '#barcelona', '#bart', '#bayofislands', '#beautiful', '#bedimages', '#bell', '#beringmy', '#bettybooppose', '#bff', '#big', '#bigbertha', '#bigbreakfast', '#blackhat', '#blessedmorethanicanimagine', '#blessedsunday', '#blogtourambiente', '#bluemountains', '#bonekachika', '#boomtaob', '#booyaa', '#bored', '#boredom', '#bradersisterhood', '#breaktime', '#breedingground', '#bringithomemy', '#brooksengland', '#burgers'

### Question 2 (1.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing the **MaxMatch algorithm** discussed in class.

NLTK has a list of words that you can use for matching, see starter code below (`words`). Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words (`words`) includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching (provided by the function `lemmatize(word)`). Note that the list of words (`words`) is the only source that you'll use for matching (i.e. you do not need to find  other external word lists). If you are unable to make any longer match, your code should default to matching a single letter.

For example, given "#newrecords", the algorithm should produce: \["#", "new", "records"\].

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [5]:
from nltk.corpus import wordnet
nltk.download('words')
nltk.download('wordnet')

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
words = set([ word.lower() for word in words ]) #lowercase all the words for better matching

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

# Implementation of MaxMatch algorithm
def max_match(hashtag):
    if len(hashtag) == 0:
        return []
    for i in range(len(hashtag), 0, -1):
        # Start with the longest string from i
        candidate = lemmatize(hashtag[:i])
        if candidate in words:
            # Continue performing MaxMatch from i+1
            return [hashtag[:i]] + max_match(hashtag[i:])
    # No matching result from the first character
    return [hashtag[0]] + max_match(hashtag[1:])

def tokenize_hashtags(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    res = {}
    for hashtag in hashtags:
        res[hashtag] = max_match(hashtag.lower())
    return res
    ###
    # Your answer ENDS HERE
    ###

#tokenise hashtags with MaxMatch
tokenized_hashtags = tokenize_hashtags(hashtags)

#print results
for k, v in sorted(tokenized_hashtags.items())[-30:]:
    print(k, v)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\cjhm0\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cjhm0\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'vega', 'n']
#veganfood ['#', 'vega', 'n', 'food']
#vegetables ['#', 'vegetables']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'vs', 'c', 'o']
#vscocam ['#', 'vs', 'coca', 'm']
#walking ['#', 'walking']
#watch ['#', 'watch']
#weare90s ['#', 'wear', 'e', '9', '0', 's']
#wearesocial ['#', 'weares', 'o', 'c', 'i', 'al']
#white ['#', 'white']
#wings ['#', 'wings']
#wok ['#', 'wo', 'k']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mates']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'ama', 's']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'ws', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [6]:
assert(len(tokenized_hashtags) == len(hashtags))
assert(tokenized_hashtags["#newrecord"] == ["#", "new", "record"])

### Question 3 (1.0 mark)

**Instructions**: Our next task is to tokenize the hashtags again, but this time using a **reversed version of the MaxMatch algorithm**, where matching begins at the end of the hashtag and progresses backwards (e.g. for <i>#helloworld</i>, we would process it right to left, starting from the last character <i>d</i>). Just like before, you should use the provided word list (`words`) for word matching.

**Task**: Complete the `tokenize_hashtags_rev(hashtags)` function by the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [7]:
# Implementation of reversed MaxMatch algorithm
def max_match_rev(hashtag):
    if len(hashtag) == 0:
        return []
    for i in range(len(hashtag)):
        # Start with the longest string from the end
        candidate = lemmatize(hashtag[i:])
        if candidate in words:
            # Continue performing MaxMatch from i+1
            return max_match_rev(hashtag[:i]) + [hashtag[i:]]
    # No matching result from the last character
    return max_match_rev(hashtag[:-1]) + [hashtag[-1]]

def tokenize_hashtags_rev(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    res = {}
    for hashtag in hashtags:
        res[hashtag] = max_match_rev(hashtag.lower())
    return res
    ###
    # Your answer ENDS HERE
    ###

#tokenise hashtags with the reversed version of MaxMatch
tokenized_hashtags_rev = tokenize_hashtags_rev(hashtags)

#print results
for k, v in sorted(tokenized_hashtags_rev.items())[-30:]:
    print(k, v)

#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'v', 'e', 'gan']
#veganfood ['#', 'v', 'e', 'gan', 'food']
#vegetables ['#', 'vegetables']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'vs', 'c', 'o']
#vscocam ['#', 'vs', 'c', 'o', 'cam']
#walking ['#', 'walking']
#watch ['#', 'watch']
#weare90s ['#', 'we', 'are', '9', '0', 's']
#wearesocial ['#', 'we', 'are', 'social']
#white ['#', 'white']
#wings ['#', 'wings']
#wok ['#', 'w', 'ok']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mates']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'a', 'mas']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'ws', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [8]:
assert(len(tokenized_hashtags_rev) == len(hashtags))
assert(tokenized_hashtags_rev["#newrecord"] == ["#", "new", "record"])

### Question 4 (1.0 mark)

**Instructions**: The two versions of MaxMatch will produce different results for some of the hashtags. For a hashtag that has different results, our task here is to use a **unigram language model** (lecture 3) to score them to see which is better. Recall that in a unigram language model we compute P(<i>#</i>, <i>hello</i>, <i>world</i> = P(<i>#</i>)\*P(<i>hellow</i>)\*P(<i>world</i>).

You should: (1) use the NLTK's Brown corpus (`brown_words`) for collecting word frequencies (note: the words are already tokenised so no further tokenisation is needed); (2) lowercase all words in the corpus; (3) use add-one smoothing when computing the unigram probabilities; and (4) work in the log space to prevent numerical underflow.

**Task**: Build a unigram language model with add-one smoothing using the word counts from the Brown corpus. Iterate through the hashtags, and for each hashtag where MaxMatch and reversed MaxMatch produce different results, print the following: (1) the hashtag; (2) the results produced by MaxMatch and reversed MaxMatch; and (3) the log probability of each result as given by the unigram language model. Note: you **do not** need to print the hashtags where MaxMatch and reversed MaxMatch produce the same results.

An example output:
```
1. #abcd
MaxMatch = [#, a, bc, d]; LogProb = -2.3
Reversed MaxMatch = [#, a, b, cd]; LogProb = -3.5

2. #efgh
MaxMatch = [#, ef, g, h]; LogProb = -4.2
Reversed MaxMatch = [#, e, fgh]; LogProb = -3.1

```

Have a look at the output, and see if the sequences with better language model scores (i.e. less negative) are generally more coherent.

In [9]:
nltk.download('brown')
from nltk.corpus import brown
from math import log

#words from brown corpus
brown_words = brown.words()

###
# Your answer BEGINS HERE
###
# Lowercase all words in the corpus
corpus = [word.lower() for word in brown_words]

# Prepare for add-one smooting
total_cnt = len(corpus) + len(set(corpus))
word_cnt_dict = defaultdict(lambda: 1)
for word in corpus:
    word_cnt_dict[word] += 1

# Implementation of computing unigram probabilities
def get_unigram_prob(tokens):
    res = 0
    for token in tokens:
        # Work in the log space to prevent numerical underflow
        res += log(word_cnt_dict[token] / total_cnt)
    return res

# Compare two versions of MaxMatch
cnt = 1
for hashtag in hashtags:
    forward = max_match(hashtag)
    reverse = max_match_rev(hashtag)
    if forward != reverse:
        forward_prob = get_unigram_prob(forward)
        reverse_prob = get_unigram_prob(reverse)
        print(f"\n{cnt}. {hashtag}\n")
        cnt += 1
        print(f"MaxMatch = {forward}; LogProb = {round(forward_prob, 2)}\n")
        print(f"Reversed MaxMatch = {reverse}; LogProb = {round(reverse_prob,2)}\n")
###
# Your answer ENDS HERE
###

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\cjhm0\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!



1. #nevergetsold

MaxMatch = ['#', 'never', 'gets', 'old']; LogProb = -38.78

Reversed MaxMatch = ['#', 'never', 'get', 'sold']; LogProb = -38.99


2. #mtlnewtech

MaxMatch = ['#', 'm', 't', 'l', 'newt', 'e', 'c', 'h']; LogProb = -88.14

Reversed MaxMatch = ['#', 'm', 't', 'l', 'new', 'tech']; LogProb = -64.42


3. #peperoni

MaxMatch = ['#', 'pep', 'er', 'on', 'i']; LogProb = -52.67

Reversed MaxMatch = ['#', 'pep', 'e', 'ro', 'ni']; LogProb = -66.32


4. #rainorshine

MaxMatch = ['#', 'rain', 'ors', 'hin', 'e']; LogProb = -62.06

Reversed MaxMatch = ['#', 'r', 'ai', 'nor', 'shine']; LogProb = -58.61


5. #bringithomemy

MaxMatch = ['#', 'bring', 'it', 'home', 'my']; LogProb = -42.4

Reversed MaxMatch = ['#', 'brin', 'git', 'home', 'my']; LogProb = -54.93


6. #focusateneo

MaxMatch = ['#', 'focus', 'aten', 'e', 'o']; LogProb = -59.31

Reversed MaxMatch = ['#', 'fo', 'c', 'u', 'sate', 'neo']; LogProb = -76.6


7. #vegan

MaxMatch = ['#', 'vega', 'n']; LogProb = -38.36

Reversed MaxMa

# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict the country of origin of a given tweet. The task here is to create training, development and test partitions from the preprocessed data (`x_processed`) and convert the bag-of-words representation into feature vectors.

**Task**: Create training, development and test partitions with a 70%/15%/15% ratio. Remember to preserve the ratio of the classes for all your partitions. That is, say we have only 2 classes and 70% of instances are labelled class A and 30% of instances are labelled class B, then the instances in training, development and test partitions should also preserve this 7:3 ratio. You may use sklearn's builtin functions for doing data partitioning.

Next, turn the bag-of-words dictionary of each tweet into a feature vector. You may also use sklearn's builtin functions for doing this (but if you don't want to use sklearn that's fine).

You should produce 6 objects: `x_train`, `x_dev`, `x_test` which contain the input feature vectors, and `y_train`, `y_dev` and `y_test` which contain the labels.

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

x_train, x_dev, x_test = None, None, None
y_train, y_dev, y_test = None, None, None

###
# Your answer BEGINS HERE
###
# Split the data into 85% training and 15% test
x_train_raw, x_test, y_train_raw, y_test = train_test_split(x_processed, y_processed, test_size=0.15, stratify=y_processed)

# Split the training set into 70% training and 15% development
x_train, x_dev, y_train, y_dev = train_test_split(x_train_raw, y_train_raw, test_size=(15/85), stratify=y_train_raw)

# Turn the bag-of-words dictionary of each tweet into a feature vector
vectorizer = DictVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_dev = vectorizer.transform(x_dev)
x_test = vectorizer.transform(x_test)
###
# Your answer ENDS HERE
###

### Question 6 (1.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation hyper-parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the **accuracy** with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output.

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance (accuracy) for different hyper-parameter settings.

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

###
# Your answer BEGINS HERE
###
# Naive Bayes
alpha_list = [0.01, 0.1, 0.5, 1.0, 2.0, 10.0]
optimal_alpha = None
optimal_accuracy_nb = 0

print("Classifier = Naive Bayes\n")
for alpha in alpha_list:
    classifier = MultinomialNB(alpha=alpha)
    classifier.fit(x_train, y_train)
    predictions = classifier.predict(x_dev)
    accuracy = accuracy_score(y_dev, predictions)
    print(f"Alpha = {alpha}, Accuracy = {round(accuracy, 2)}")
    
    if accuracy > optimal_accuracy_nb:
        optimal_accuracy_nb = accuracy
        optimal_alpha = alpha

print(f"\nOptimal Alpha = {optimal_alpha}, Accuracy = {round(optimal_accuracy_nb, 2)}\n\n")


# Logistic Regression
c_list = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
optimal_c = None
optimal_accuracy_lr = 0

print("\nClassifier = Logistic Regression\n")
for c in c_list:
    classifier = LogisticRegression(C=c)
    classifier.fit(x_train, y_train)
    predictions = classifier.predict(x_dev)
    accuracy = accuracy_score(y_dev, predictions)
    print(f"C = {c}, Accuracy = {round(accuracy, 2)}")

    if accuracy > optimal_accuracy_lr:
        optimal_accuracy_lr = accuracy
        optimal_c = c

print(f"\nOptimal C = {optimal_c}, Accuracy = {round(optimal_accuracy_lr, 2)}")
###
# Your answer ENDS HERE
###

Classifier = Naive Bayes

Alpha = 0.01, Accuracy = 0.3
Alpha = 0.1, Accuracy = 0.3
Alpha = 0.5, Accuracy = 0.3
Alpha = 1.0, Accuracy = 0.28
Alpha = 2.0, Accuracy = 0.27
Alpha = 10.0, Accuracy = 0.27

Optimal Alpha = 0.01, Accuracy = 0.3



Classifier = Logistic Regression

C = 0.001, Accuracy = 0.21
C = 0.01, Accuracy = 0.21
C = 0.1, Accuracy = 0.24
C = 1.0, Accuracy = 0.29
C = 10.0, Accuracy = 0.28
C = 100.0, Accuracy = 0.27

Optimal C = 1.0, Accuracy = 0.29


### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both **accuracy** and **macro-averaged F-score** for each classifier. Be sure to label your output. You may use sklearn's inbuilt functions.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using their optimal hyper-parameter settings based on their development performance.

In [12]:
###
# Your answer BEGINS HERE
###
from sklearn.metrics import f1_score

# Naive Bayes
classifier_nb = MultinomialNB(alpha=optimal_alpha)
classifier_nb.fit(x_train, y_train)
predictions_nb = classifier_nb.predict(x_test)
accuracy_nb = accuracy_score(y_test, predictions_nb)
f1_nb = f1_score(y_test, predictions_nb, average='macro')

print("Classifier = Naive Bayes")
print(f"Accuracy = {accuracy_nb}")
print(f"Macro-averaged F-score = {f1_nb}\n")

# Logistic Regression
classifier_lr = LogisticRegression(C=optimal_c)
classifier_lr.fit(x_train, y_train)
predictions_lr = classifier_lr.predict(x_test)
accuracy_lr = accuracy_score(y_test, predictions_lr)
f1_lr = f1_score(y_test, predictions_lr, average='macro')

print("Classifier = Logistic Regression")
print(f"Accuracy = {accuracy_lr}")
print(f"Macro-averaged F-score = {f1_lr}")
###
# Your answer ENDS HERE
###

Classifier = Naive Bayes
Accuracy = 0.323943661971831
Macro-averaged F-score = 0.3139035152219737

Classifier = Logistic Regression
Accuracy = 0.34507042253521125
Macro-averaged F-score = 0.32701543815478307


### Question 8 (1.0 mark)

**Instructions**: Print the most important features and their weights for each class for the two classifiers.


**Task**: For each of the classifiers (Logistic Regression and Naive Bayes) you've built in the previous question, print out the top-20 features (words) with the highest weight for each class (countries).

An example output:
```
Classifier = Logistic Regression

Country = au
aaa (0.999) bbb (0.888) ccc (0.777) ...

Country = ca
aaa (0.999) bbb (0.888) ccc (0.777) ...

Classifier = Naive Bayes

Country = au
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...

Country = ca
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...
```

Have a look at the output, and see if you notice any trend/pattern in the words for each country.

In [13]:
###
# Your answer BEGINS HERE
###
import numpy as np

top = 20

def print_features(classifier):
    features = np.array(vectorizer.get_feature_names_out())
    for i, country in enumerate(classifier.classes_):
        # Extract all features' weights based on types of classifiers
        feature_weights = None
        if classifier == classifier_nb:
            feature_weights = classifier.feature_log_prob_[i]
        elif classifier == classifier_lr:
            feature_weights = classifier.coef_[i]
        else:
            print("Error: Classifier Not Found!")

        # Extract top 20 features and corresponding weights
        top_features_indices = feature_weights.argsort()[-top:][::-1]
        top_features = features[top_features_indices]
        top_weights = feature_weights[top_features_indices]

        print(f"\nCountry = {country}")
        for feature, weight in zip(top_features, top_weights):
            print(f"{feature} ({round(weight, 2)})", end=" ")
        print("\n")
###
# Your answer ENDS HERE
###

In [14]:
print(f"Classifier = Naive Bayes\nTop {top} features for each class")
print_features(classifier_nb)

Classifier = Naive Bayes
Top 20 features for each class

Country = au
i'm (-4.42) melbourne (-4.75) one (-4.75) great (-4.98) little (-4.98) love (-5.26) make (-5.26) australia (-5.26) even (-5.26) keep (-5.26) @micksunnyg (-5.67) hey (-5.67) brilliant (-5.67) would (-5.67) vca (-5.67) good (-5.67) anything (-5.67) look (-5.67) though (-5.67) #mtvhottest (-5.67) 


Country = ca
i'm (-4.66) one (-4.84) like (-5.06) great (-5.06) new (-5.06) maybe (-5.06) good (-5.06) life (-5.35) today (-5.35) first (-5.35) next (-5.35) going (-5.35) right (-5.35) get (-5.35) made (-5.35) learning (-5.35) school (-5.35) day (-5.75) would (-5.75) need (-5.75) 


Country = de
love (-4.03) posted (-4.25) photo (-4.53) night (-4.53) i'm (-4.94) please (-4.94) come (-4.94) painting (-4.94) could (-4.94) miss (-4.94) germany (-4.94) #watch (-5.63) #thankful (-5.63) main (-5.63) luxembourg (-5.63) https://t.co/brkwmsvzrb (-5.63) https://t.co/agtyffnl4h (-5.63) tickets (-5.63) flatweave (-5.63) #walking (-5.63)

In [15]:
print("Classifier = Logistic Regression\nTop 20 features for each class")
print_features(classifier_lr)

Classifier = Logistic Regression
Top 20 features for each class

Country = au
little (1.1) melbourne (0.99) #melbourne (0.92) australia (0.83) @micksunnyg (0.83) hey (0.81) brilliant (0.8) beach (0.68) spammed (0.63) instagrams (0.63) away (0.62) @dasheryoung (0.62) lachie (0.62) keep (0.58) http://t.co/blhj9cmxit (0.57) @christorrano (0.57) geelong (0.57) @shell_07 (0.57) freo (0.57) song (0.56) 


Country = ca
happen (0.86) hate (0.82) next (0.81) learning (0.78) life (0.76) works (0.72) sounds (0.7) one (0.69) movies (0.69) healthy (0.66) @cheetahbiatch (0.62) xoxo (0.62) thing (0.61) @lola9793 (0.6) cutest (0.6) @samanthasharris (0.6) second (0.59) school (0.58) joking (0.58) good (0.58) 


Country = de
posted (1.03) love (0.99) night (0.82) @fabiomarabini (0.78) happened (0.74) gauting (0.67) hyde (0.67) #utopia (0.67) #truegrip (0.67) https://t.co/brkwmsvzrb (0.67) jessica (0.67) could (0.65) germany (0.65) painting (0.64) workout (0.63) https://t.co/i7j5pmd3mx (0.63) photo (0.63